In [1]:
import torch.nn.functional as F
import torch.nn as nn
import torch

from tqdm import tqdm, trange

N = 1000
batch_size = 32

# adjust this parameter to make GPU memory low (e.g. 30% free)
seq_len = 1200

class TestModel(nn.Module):
    def __init__(self):
        super(TestModel, self).__init__()
        
        self.lstm = nn.LSTM(seq_len, seq_len, batch_first=True)
        self.fc = nn.Linear(seq_len, 1)
    
    def forward(self, x):
        x = F.one_hot(x, seq_len).float()
        x, _ = self.lstm(x)
        x = self.fc(x)
        return x.squeeze(-1)

In [2]:
x = torch.randint(0, seq_len, [N, seq_len]).long()
y = torch.randint(0, seq_len, [N, seq_len]).float()

In [3]:
model = TestModel().cuda()

for i in trange(0, N, batch_size):
    x_batch = x[i:i+batch_size].cuda()
    y_batch = y[i:i+batch_size].cuda()

    y_pred = model(x_batch)

    with torch.no_grad():
        # if using a RTX 30 series GPU and with this line, a CUDA error may be raised
        F.binary_cross_entropy_with_logits(y_pred.detach(), y_batch)
        pass

 19%|█▉        | 6/32 [00:01<00:08,  3.16it/s]


RuntimeError: CUDA error: unspecified launch failure